# Neural Networks for Structured Data

** *Based on Murphy 2022 (Chapter 13) 

## Introduction (13.1)
 

- Logistic and linear regression assume that the input-output mapping is linear
- We can make these models more flexible by performing feature transformation (aka basis function expansion):

$
\begin{aligned}
f(x;\theta) = W\phi(x) + b
\end{aligned}
$

- The model is still linear in the parameters $\theta = (W,b)$, so model fitting is easy since the negative log likelihood is convex. However, we still have to specify the feature transofrmation $\phi(x)$ by hand. 

- Instead, we can endow the frature extractor with its own parameters:

$
\begin{aligned}
f(x;\theta) = W\phi(x;\theta) + b
\end{aligned}
$

Now $\theta = (\theta_1, \theta_2)$, where $\theta_1$ are the parameters of the linear model ($\theta_1 = (W,b)$), and $\theta_2$ are the parameters of the feature extractor. 

We can repeat this recursively to create more complex functions:

$
\begin{aligned}
f(x;\theta) = f_{L}(f_{L-1}(...(f_{}(x))...))
\end{aligned}
$

Where $f_l = f(x,\theta_l)$

**This is the key idea behind deep neural networsks (DNNs)**

- DNN exmplass a larger family of model sin which we compose differentiable functions into any kind of directed acyclic graph (DAG) mapping input to output. 

- $f(x;\theta) = f_{L}(f_{L-1}(...(f_{}(x))...))$ is the simplest example where DAG is a chain, and this is known as a **feedforward neural network** of **multilayer perceptron (MLP)**

- An MLP assumes that the input is a fixed-dimensional vector, and it is common to call such data **structured data** or **tabular data**. 

## Multilayer perceptrons (13.2)

- Perceptron is a deterministic version of logistic regression. It a mapping of the following form:

$
\begin{aligned}
f(x:\theta) = \mathbb{I}(w^Tx + b \ge0) = H(w^Tx + b)
\end{aligned}
$

- $H(A)$ is a heavside step function, also known as a linear threshold function. Since decision boundaries represented by perceptrons are linear, they are very limited in what they can represent. This was noted by Minsy & Papert (1969). 

- To solve the XOR problem we need a **multilayer perceptron (MLP)**. We add hidden units and thus have multiple layers. Then MLPs can solve problems which are not linearly separable. 





### Differentiable MLPs

- The Havside function is not differentiable, so we can't train the model. 

- Thus we use a differentiable **activation function** $\phi: \mathbb{R} \rightarrow \mathbb{R}$. Hidden units $z_{l}$ at each layer are linear transformation of the hidden units at the previous layer passed through the activation function:

$
\begin{aligned}
z_{l} = \phi-{l}(b_{l} + W_{l}z_{l-1})
\end{aligned}
$

or in scalar form:

$
\begin{aligned}
z_{k,l} = \phi_{l}(b_{kl} + \sum_{j=1}^{K_{l-1}}W_{jkl}z_{jl-1})
\end{aligned}
$

- The quantity passed to the activation function is called the **pre-activations**:

$
\begin{aligned}
a_{l} = b_{l} + W_{l}z_{l-1}
\end{aligned}
$

- so:

$
\begin{aligned}
z_{l} = \phi_{l}(a_{l})
\end{aligned}
$

- If we componse L of these fuctnions together we can compute the gradient of the output wrt the parameters in each layer using the cahing rule, also non as **backpropagation**.



### Activation functions

- If we use a linear activation function $\phi_{l}(a) = c_{l}a$, then the model reduces to a regular linear model.

- In the early days of neural networks the common nonlinear activation function was the sigmoid which is a smooth approximation to the Heaviside function:

$
\begin{aligned}
\phi(a) = \sigma(a) = \frac{1}{1+e^{-a}}
\end{aligned}
$

- The sigmoid saturates at 0 & 1. We can also use the tanh (-1 1). For these functions, in the saturated regimes, the gradient of the output wrt the input will be close to zeo, so any gradient signal from higher layers will not be able to propagate back to earlier layes. This is the **vanishing gradient problem** making it hard to rain the model using gradient descent. 

- The most common solution is the use of teh **rectified linear unit (ReLU)**:

$
\begin{aligned}
ReLU(a) = \max(a,0) = a\mathbb{I}(a \ge 0)
\end{aligned}
$

- The ReLU function turns off negative inputs and passes positive inputs unchanged. 

Play around with the activation functions here: https://playground.tensorflow.org/

- MLP with one hidden layer is a **universal function approximator**. It can model any suitably smooth function given enough hidden unites to an arbitrary degree of accuracy. Each hidden unit can specify half plane, so a sufficiantly large comibnation of these can carve up any region of space. 

- However, deep netwroks work better than shallow ones. THe later layers can levarage the features that are learned by earlier layers, and the function is defined in a compositional or hierarchical manner.

## Backpropagation (13.3)

- Backpropagation algorithm - used to compute the gradient of a loss functoin applied to the output of the network wrt the parameters in each layer


### Forward vs. reverse mode differentiation (13.3.1)

- We first consider a linear chain of stacked layers (as in an MLP). 
- In this case backprop is equivalent to repeated application of the chain rule of calculus:

$
\begin{aligned}
\frac{d}{dx}f(u(x)) = \frac{du}{dx}\frac{df(u)}{du}
\end{aligned}
$


- Consider the mapping:

$
\begin{aligned}
o = f(x)
\end{aligned}
$

$
\begin{aligned}
x \in \mathbb{R}^n
\end{aligned}
$

$
\begin{aligned}
o \in \mathbb{R}^m
\end{aligned}
$

- Where:

$
\begin{aligned}
f = f_4 \circ f_3 \circ f_2 \circ f_1
\end{aligned}
$


- We can compute the gradient of the output wrt the input using the chain rule:

$
\begin{aligned}
\frac{\partial o}{\partial x} = \frac{\partial o}{\partial x_4} \frac{\partial x_4}{\partial x_3} \frac{\partial x_3}{\partial x_2} \frac{\partial x_2}{\partial x} = \frac{\partial f_4(x_4)}{\partial x_4} \frac{\partial f_3(x_3)}{\partial x_3} \frac{\partial f_2(x_2)}{\partial x_2} \frac{\partial f_1(x)}{\partial x} = J_{f_4}(x_4)J_{f_3}(x_3)J_{f_2}(x_2)J_{f_1}(x)
\end{aligned}
$

- Jacobian $J_f(x)$ can be computed efficiently. 

$
\begin{aligned}
J_f(x) = \frac{\partial f(x)}{\partial x} = 
\begin{bmatrix}
\frac{\partial f_1(x)}{\partial x_1} & \frac{\partial f_1(x)}{\partial x_2} & \dots & \frac{\partial f_1(x)}{\partial x_n} \\
\frac{\partial f_2(x)}{\partial x_1} & \frac{\partial f_2(x)}{\partial x_2} & \dots & \frac{\partial f_2(x)}{\partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial f_m(x)}{\partial x_1} & \frac{\partial f_m(x)}{\partial x_2} & \dots & \frac{\partial f_m(x)}{\partial x_n} \\
\end{bmatrix} = \begin{bmatrix}
\frac{\partial f_1(x)}{\partial x} \\
\frac{\partial f_2(x)}{\partial x} \\
\vdots \\
\frac{\partial f_m(x)}{\partial x} \\
\end{bmatrix} = \begin{bmatrix}
\nabla f_1(x)^T \\
\nabla f_2(x)^T \\
\vdots \\
\nabla f_m(x)^T \\
\end{bmatrix} = \begin{bmatrix}
\frac{\partial f(x)}{\partial x_1} & \frac{\partial f(x)}{\partial x_2} & \dots & \frac{\partial f(x)}{\partial x_n} \\
\end{bmatrix}
\end{aligned}
$

- Note that the gradient $\nabla f(x)$ is a column vector, awhile the Jacobian $J_f(x)$ is a row vector. Thus we technically have:

$
\begin{aligned}
\nabla f(x)^T = J_f(x)
\end{aligned}
$

**Vector-Jacobian product (VJP)**
- We can extract the i'th rom from $J_f(x)$ by using a vector Jacobian product of the form: $e_i^TJ_f(x)$ where $e_i$ is the i'th unit basis vector ($e_i \in \mathbb{R}^m$).

**Jacobian-vector product (JVP)**
- We can extract the j'th column from $J_f(x)$ by using a Jacobian vector product of the form: $J_f(x)e_j$ where $e_j \in \mathbb{R}^n$.

- Thus, the computation of $J_f(x)$ reduces to either $n$ JVPs or $m$ VJPs.

**When $n < m$**

- It is more efficient to compute the JVPs than the VJPs.
- This is **forward mode differentiation**.

$
\begin{aligned}
J_f(x)v = J_{f_4}(x_4)J_{f_3}(x_3)J_{f_2}(x_2)J_{f_1}(x_1)v 
\end{aligned}
$

**When $n > m$**

- It is more efficient to compute the VJPs than the JVPs.
- This is **reverse mode differentiation**.

$
\begin{aligned}
u^TJ_f(x) = u^TJ_{f_4}(x_4)J_{f_3}(x_3)J_{f_2}(x_2)J_{f_1}(x_1)
\end{aligned}
$